In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/lora-overlap/adapter_model.safetensors
/kaggle/input/lora-overlap/adapter_config.json
/kaggle/input/lora-overlap/tokenizer.json
/kaggle/input/lora-overlap/tokenizer_config.json
/kaggle/input/lora-overlap/special_tokens_map.json
/kaggle/input/onet-test-set/onet_test.jsonl


In [2]:
import torch
print(torch.cuda.is_available())  # ควรได้ True
print(torch.cuda.device_count())  # ควร ≥ 1
print(torch.cuda.get_device_name(0))  # ชื่อการ์ดจอ

True
2
Tesla T4


In [3]:
# df = pd.read_csv('/kaggle/input/onet-data/math_problems_gemini_flash_latex.csv')
df = pd.read_json('/kaggle/input/onet-test-set/onet_test.jsonl', lines=True)

In [4]:
df_math = df[df['subject'] == 'math']
df_math = df_math.drop(columns=['year','no','subject'])

dataset = df_math.values.tolist()
df_math

,question,a,b,c,d,e,answer
111,"กำหนดให้ x , y , z เป็นจำนวนจริงที่ไม่เท่ากับ...",xz < 0,xz > 0,xy > 0,yz < 0,yz > 0,b
112,กำหนดให้ p และ q เป็นประพจน์ \nข้อใดต่อไปนี้ ส...,(pvq) → q,(~p^q) → q,(~q^q) → p,p v (p → q),(pvq) (~p^~q),a
113,"กำหนดให้ U = {1,2,3,...,50} และ\n ...",8,9,15,17,19,d
114,ในการแข่งขันกีฬาสามประเภท ของโรงเรียนบ้านโกงกา...,13 คน,16 คน,17 คน,19 คน,23 คน,c
115,ตามสถิติของการเกิดโรคระบาดครั้งหนึ่งเมือง ก มี...,285 คน,327 คน,334 คน,351 คน,373 คน,b
116,"นาวาเปิดบัญชีเงินฝากด้วยเงินต้น 10,000 บาทและไ...",400 [(3/2)^1/20 - 1] เปอร์เซ็นต์,4[(2/3)^1/20 + 1] เปอร์เซ็นต์,03[(3/2)^1/15 + 1] เปอร์เซ็นต์,400[1-(2/3)1/20] เปอร์เซ็นต์,300[(3/2)1/15 - 1] เปอร์เซ็นต์,a
117,"บริษัทนําเที่ยวแห่งหนึ่งซื้อรถมินิบัสราคา 2,00...",10 ปี,11 ปี,12 ปี,13 ปี,14 ปี,d
118,คะแนนสอบวิชาคณิตศาสตร์ จากคะแนนเต็ม 100 คะแนนข...,2 คะแนน,18 คะแนน,20 คะแนน,21 คะแนน,22 คะแนน,e
119,เด็กคนหนึ่งนำภาชนะที่แตกต่างกันทั้งหมด 7 ใบประ...,1/840,1/210,3/70,1/35,2/35,e
120,มีส้ม ฝรั่ง แอปเปิ้ล มะละกอ สาลี่ แก้วมังกร มะ...,280,456,8!,"C8,3 + C5,4","C8,3 x C8,4",a


In [5]:
from peft import PeftModel, PeftConfig
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "scb10x/llama3.2-typhoon2-3b-instruct"

path = '/kaggle/input/lora-overlap'

base_model = AutoModelForCausalLM.from_pretrained(model_id)
inference_tokenizer = AutoTokenizer.from_pretrained(path)
base_model.resize_token_embeddings(len(inference_tokenizer))

inference_model = PeftModel.from_pretrained(base_model, path)

2025-05-09 05:30:45.748227: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746768645.938041      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746768645.991780      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/778 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/303 [00:00<?, ?B/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


In [6]:
# from peft import PeftModel, PeftConfig
# from transformers import AutoTokenizer, AutoModelForCausalLM

# model_id = "scb10x/llama3.2-typhoon2-3b-instruct"


# inference_model = AutoModelForCausalLM.from_pretrained(model_id)
# inference_tokenizer = AutoTokenizer.from_pretrained(model_id)


In [7]:
from torch import nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
inference_model = inference_model.to(device)
inference_model = nn.DataParallel(inference_model)

In [8]:
def safe_add_token(tokenizer, token):
    if token not in tokenizer.get_vocab():
        tokenizer.add_special_tokens({'additional_special_tokens': [token]})
        print(f"Added token: {token}")
    else:
        print(f"Token already exists: {token}")

In [9]:
import torch

def generate_with_budget_forcing(model,
                                 tokenizer,
                                 messages,
                                 min_budget=512,
                                 max_budget=768,
                                 chunk_size =400,
                                 special_token_bias=5.0):
    # หากใช้ DataParallel ต้องอ้างถึง .module
    base_model = model.module if isinstance(model, torch.nn.DataParallel) else model
    device = next(base_model.parameters()).device
    
    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(device)

    

    thinking_end_token = "<End of thinking>"
    wait_token = "Wait,"

    safe_add_token(tokenizer, thinking_end_token)
    safe_add_token(tokenizer, wait_token)
    base_model.resize_token_embeddings(len(tokenizer))

    thinking_end_id = tokenizer.convert_tokens_to_ids(thinking_end_token)
    wait_id = tokenizer.convert_tokens_to_ids(wait_token)

    generated_ids = input_ids
    thinking_trace = ""

    base_model.eval()
    with torch.no_grad():
        for step in range(max_budget):
            if step % 50 == 0:
                print('------------\n')
                print(f'token:{step}\n')
                print(tokenizer.decode(generated_ids[0, -50:]))

            if step % chunk_size == 0 and step > 0:
                mark_as_prev_ids = torch.tensor(tokenizer.encode(f"\nPrevious reasoning from Step {step//chunk_size}:\n"), device=device).unsqueeze(0)
                generated_ids = torch.cat((input_ids, mark_as_prev_ids, generated_ids[:, -100:]), dim=1)

            attention_mask = (generated_ids != tokenizer.pad_token_id).long()
            outputs = model(input_ids=generated_ids, attention_mask=attention_mask)
            logits = outputs.logits[:, -1, :]
            logits[:, thinking_end_id] += special_token_bias
            isWait = False

            if step < min_budget:
                new_token = torch.argmax(logits, dim=-1)
                if new_token.item() in [thinking_end_id, tokenizer.eos_token_id]:
                    logits[:, thinking_end_id] = -1e9  # ใช้ -1e9 แทน -inf
                    logits[:, wait_id] = 1e9
                    isWait = True
                    print("Wait, ", tokenizer.decode(new_token))

            if step >= max_budget - 1:
                print('reach max budget')
                logits[:, thinking_end_id] = 1e9

            next_token = torch.argmax(logits, dim=-1)

            if next_token.item() == wait_id:
                print('wait token', tokenizer.decode(next_token))

            generated_ids = torch.cat((generated_ids, next_token.unsqueeze(-1)), dim=1)
            if isWait:
                revise_ids = torch.tensor(tokenizer.encode("Revise your thought each step.\n"), device=device).unsqueeze(0)
                generated_ids = torch.cat((generated_ids, revise_ids), dim=1)

            thinking_trace += tokenizer.decode(next_token)

            if next_token.item() == thinking_end_id:
                print('found thinking_end_token:', tokenizer.decode(next_token))
                break

        summary_prompt_ids = tokenizer.encode(
            "\nYou already finished reasoning. Just summarize the answer.\n[My choice:",
            return_tensors="pt"
        ).to(device)
        
        response_input = torch.cat((generated_ids, summary_prompt_ids), dim=1)

        answering_trace = '[My choice:'
        for _ in range(10):
            outputs = model(response_input)
            logits = outputs.logits[:, -1, :]  # logits ของ token ล่าสุด
            next_token = torch.argmax(logits, dim=-1)  # ใช้ argmax แทน multinomial
            response_input = torch.cat((response_input, next_token.unsqueeze(-1)), dim=1)
        
            if next_token.item() == tokenizer.eos_token_id:
                print('generate eos')
                break
        
            answering_trace += tokenizer.decode(next_token)

    output_text = tokenizer.decode(response_input[0], skip_special_tokens=True)
    print('------output-------')
    print(output_text)
    print('------answer-------')
    print(answering_trace)
    return answering_trace

In [10]:
dataset[0]

[' กำหนดให้ x , y , z เป็นจำนวนจริงที่ไม่เท่ากับศูนย์และ sqrt(x^2 y^4 z^2 = - xy^2z  ข้อใดต่อไปนี้เป็นไปไม่ได้',
 'xz < 0',
 'xz > 0',
 'xy > 0',
 'yz < 0',
 'yz > 0',
 'b']

In [11]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [12]:
import torch
import gc

choice = ['a','b','c','d','e']
model_ans = []
for e in dataset:
    # prompt = 'Example-คำถาม:1x1 = ?\nข้อ ก)1\nข้อ ข)2\nข้อ ค)3\nThinking:\nstep1:Need to multiply 1 with 1 \nstep2:1x1=1 \nFinish Thinking<End of thinking>\nMy Choice: ข้อ ก)\n'
    prompt = f'Task-จงตอบคำถามจากตัวเลือกที่กำหนดให้\nคำถาม:{e[0]}\n'
    for i in range(1,6):
        prompt += f'ข้อ {choice[i-1]}) {e[i]}\n'

    # prompt += "\nโปรดแสดงวิธีคิดอย่างมีเหตุผลในภาษาอังกฤษแบบกระชับ เมื่อคิดเสร็จให้พิมพ์คำว่า <End of thinking> ต่อท้าย และเลือกเพียงคำตอบเดียวในรูปแบบ [My Choice:ข้อX]\n\nThinking:"
    # prompt += '\n จงคิดอย่างเป็นขั้นตอนเป็นภาษาอังกฤษ และเลือกข้อที่ถูกที่สุด โดยตอบในรูปแบบของ JSON = {"Thinking":"..." , "Answer":"ข้อX" (เมื่อ X เป็นตัวเลข 1-5)} โดยให้สิ่งที่คิดอยู่ใน "Thinking" และให้ข้อที่ถูกที่สุดอยู่ใน "Answer"\n'
    # prompt += 'Your Task-จงคิดอย่างเป็นขั้นตอนในภาษาอังกฤษเพื่อตอบคำถามจาก Task เมื่อคิดเสร็จให้แสดง Finish Thinking<End of thinking> ในรูปแบบ = [Thinking:... Finish Thinking<End of thinking>] และเลือกตัวอักษรที่แทนข้อที่คำตอบถูกที่สุดเพียงข้อเดียวในรูปแบบ = [My Choice:ข้อ...] \nThinking:'
    # prompt += 'Thinking:'

    # messages = [
    # {
    #     "role": "system",
    #     "content": (
    #         "You are Typhoon, a smart AI assistant helping users solve multiple-choice math questions.\n"
    #         "Always think in English, showing clear, concise, and logical reasoning before answering.\n"
    #         "Begin with: 'Thinking: I will use N steps.' (between 5 and 10 steps).\n"
    #         "Label each step as: Step x/N. Keep each step short (under 75 tokens). Always start from Step 1/N.\n"
    #         "If you find the answer early, use the remaining steps to verify.\n"
    #         "When done, write: <End of thinking>\n"
    #         "Then on the next line, answer in this exact format: [My Choice: X] where X is one of a, b, c, d, or e.\n"
    #         "Do not continue or repeat after <End of thinking>. Do not answer before it.\n"
    #         "\n"
    #         "Example:\n"
    #         "Thinking: I will use 6 steps.\n"
    #         "Step 1/6: ...\n"
    #         "...\n"
    #         "<End of thinking>\n"
    #         "[My Choice: c]"
    #     )
    # },

    #     {"role": "user", "content": prompt},
    # ]

    messages = [
    {
        "role": "system",
        "content": (
            "First, thinking to solve problem in English or Thai step by step.\n"
            "Start with: Thinking: I will use N steps.\n"
            "Keep each step concise, logical.\n"
            "When you finish, write exactly: <End of thinking>\n"
            "Then on the next line, answer again with this format: [My Choice: X]\n"
        )
    },
    {
        "role": "user",
        "content": prompt
    }
]




    answer = generate_with_budget_forcing(inference_model,inference_tokenizer,messages)
    model_ans.append(answer)
    gc.collect()
    torch.cuda.empty_cache()

Token already exists: <End of thinking>
Token already exists: Wait,
------------

token:0

) xz < 0
ข้อ b) xz > 0
ข้อ c) xy > 0
ข้อ d) yz < 0
ข้อ e) yz > 0<|eot_id|><|start_header_id|>assistant<|end_header_id|>


------------

token:50

<|begin_of_text|>Thinking: I will use 5 steps.
Step 1/5: Alright, so I have this problem here where I need to figure out which of the given statements is impossible given that sqrt(x² y⁴ z²) = -
------------

token:100

xy²z. Let me break it down step by step.

First, I need to understand what the equation sqrt(x² y⁴ z²) = -xy²z means. This is an equation involving square roots and variables x, y
------------

token:150

, and z. Since it's a square root, the left side is non-negative, but the right side is negative. So, that tells me that the expression inside the square root must be negative, because otherwise, the equation wouldn't hold.


------------

token:200

So, let's write that out: sqrt(x² y⁴ z²) = -xy²z. Since the square root of a negative n

In [13]:
import re

def extract_choice(text):
    """
    Extract the answer choice from model output, expected in format like:
    [My Choice:ข้อX], [My choice:X], [My Choice:X], etc.
    Returns: 'a', 'b', 'c', 'd', 'e' or '-' if not found
    """
    # Normalize to lowercase to handle both uppercase and lowercase letters
    text = text.lower()

    # Try multiple patterns
    patterns = [
        r'\[my choice:ข้อ([abcde])\]',
        r'\[my choice:\s*([abcde])\]',  # without "ข้อ"
        r'\[my choice:\s*ข้อ([abcde])\]',  # with optional whitespace
    ]

    for pattern in patterns:
        match = re.search(pattern, text)
        if match:
            return match.group(1)

    return '-'

In [14]:


def evaluate_model_outputs(predictions, labels):
    """
    predictions: list of raw text outputs from model
    labels: list of correct choices (as 'a', 'b', 'c', 'd', 'e')
    Returns: accuracy (float), detailed results (list of dict)
    """
    correct = 0
    results = []

    for pred_text, true_label in zip(predictions, labels):
        pred_choice = extract_choice(pred_text)
        # print(pred_choice)

        is_correct = (pred_choice == true_label)
        if is_correct:
            correct += 1
        
        results.append({
            "prediction": pred_choice,
            "ground_truth": true_label,
            "is_correct": is_correct,
            "full_output": pred_text
        })

    accuracy = correct / len(labels)
    return accuracy, results

In [15]:
model_ans

['[My choice: e]\n</think>\n\nGiven the equation \\( \\',
 '[My choice: e]\n</think>\n\n**Final Answer**\nThe',
 '[My choice: b]<|end_of_text|><|end_of_text|><|end_of_text|>-addrum<|end_of_text|><|end_of_text|>',
 '[My choice: c]<|end_of_text|><|end_of_text|><|end_of_text|>aside<|end_of_text|><|end_of_text|>section<|end_of_text|>',
 '[My choice: c]<|end_of_text|><|end_of_text|><|end_of_text|>-entity<|end_of_text|>-entity<|end_of_text|>-entity',
 '[My choice: e]<|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|>',
 '[My choice: c]<|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|>',
 '[My choice: b<|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|>section<|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|>',
 '[My choice: b] <|end_of_text|><|end_of_text|><|end_of_text|>lege<|end_of_text|><|end_of_text|>lege',
 '[My choice: e]<|end_of_

In [16]:
correct_answers = []
for e in dataset:
    correct_answers.append(e[6])

In [17]:
correct_answers

['b',
 'a',
 'd',
 'c',
 'b',
 'a',
 'd',
 'e',
 'e',
 'a',
 'e',
 'e',
 'c',
 'c',
 'a',
 'b']

In [18]:
acc, eval_results = evaluate_model_outputs(model_ans, correct_answers)
print(f"Accuracy: {acc * 100:.2f}%")

Accuracy: 6.25%


In [19]:
import pandas as pd

df = pd.DataFrame(eval_results)
df.to_csv("evaluation_report.csv", index=False)